In [1]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-02-20 16:43:24--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 185.199.109.153, 185.199.111.153, 185.199.108.153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip’

WiC_dataset.zip     100%[===================>] 269.52K  --.-KB/s    in 0.03s   

2023-02-20 16:43:24 (8.03 MB/s) - ‘WiC_dataset.zip’ saved [275984/275984]



In [2]:
import pandas as pd
import numpy as np
import os
import io
import spacy
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [3]:
directory = os.makedirs( '/content/WIC', exist_ok=True )

In [4]:
dev = os.makedirs( '/content/WIC/dev', exist_ok=True )
train = os.makedirs( '/content/WIC/train', exist_ok=True )
test = os.makedirs( '/content/WIC/test', exist_ok=True )


In [5]:
#path = "/content/dev/dev.data.txt"

In [6]:
#directory = path.split('/')[2]

In [7]:
!pip install spacy  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
from zipfile import ZipFile
with ZipFile('WiC_dataset.zip', 'r') as WiC:
   WiC.extractall()

In [9]:
df_dev = pd.read_csv("/content/dev/dev.data.txt", sep = '\t', names=["lemma", "pos", "index", "sent1", "sent2"], encoding = 'UTF-8')
df_train = pd.read_csv("/content/train/train.data.txt", sep = '\t', names=["lemma", "pos", "index", "sent1", "sent2"], encoding = 'UTF-8')
df_test = pd.read_csv("/content/test/test.data.txt", sep = '\t', names=["lemma", "pos", "index", "sent1", "sent2"], encoding = 'UTF-8')


In [10]:
dev_label= pd.read_csv("/content/dev/dev.gold.txt", sep = '\t', names = ['judgment'], encoding = 'UTF-8')
train_label= pd.read_csv("/content/train/train.gold.txt", sep = '\t', names = ['judgment'], encoding = 'UTF-8')
test_label= pd.read_csv("/content/test/test.gold.txt", sep = '\t', names = ['judgment'], encoding = 'UTF-8')

In [11]:
x= df_dev['index'].tolist()
y = df_train['index'].tolist()
z = df_test['index'].tolist()

In [12]:
df_dash = df_dev[['lemma', 'sent1', 'sent2']]
df_dash1 = df_train[['lemma', 'sent1', 'sent2']]
df_dash2 = df_test[['lemma', 'sent1', 'sent2']]

In [13]:
df1 = df_dash[['lemma', 'sent1']]
df1.columns =  ['word', 'sent']
df1["index"] = df1.groupby("word").cumcount()*2

dfone = df_dash1[['lemma', 'sent1']]
dfone.columns =  ['word', 'sent']
dfone["index"] = dfone.groupby("word").cumcount()*2

dfein = df_dash2[['lemma', 'sent1']]
dfein.columns =  ['word', 'sent']
dfein["index"] = dfein.groupby("word").cumcount()*2

In [14]:
df2 =  df_dash[['lemma', 'sent2']]
df2.columns =  ['word', 'sent']
df2["index"] = df2.groupby("word").cumcount()*2+1

dftwo =  df_dash1[['lemma', 'sent2']]
dftwo.columns =  ['word', 'sent']
dftwo["index"] = dftwo.groupby("word").cumcount()*2+1

dfdu =  df_dash2[['lemma', 'sent2']]
dfdu.columns =  ['word', 'sent']
dfdu["index"] = dfdu.groupby("word").cumcount()*2+1

In [15]:
df_final_dev = pd.concat([df1, df2])
df_final_dev = df_final_dev.sort_values(by = ['word', 'index'], ascending = [True, True]) 

df_final_train = pd.concat([dfone, dftwo])
df_final_train = df_final_train.sort_values(by = ['word', 'index'], ascending = [True, True])

df_final_test = pd.concat([dfein, dfdu])
df_final_test = df_final_test.sort_values(by = ['word', 'index'], ascending = [True, True])

In [16]:
df_final_test

,word,sent,index
59,abort,I wasted a year of my life working on an abort .,0
59,abort,He sent a short message requesting an abort du...,1
1095,absorption,"The absorption of light , heat , electricity ,...",0
1095,absorption,The absorption of photons by atoms or molecules .,1
618,accept,To accept the report of a committee .,0
...,...,...,...
1224,zero,Write 0.0 to indicate a floating point number ...,5
1023,zinger,She tried to think of some killer of an argume...,0
1023,zinger,He always greeted me with a new zinger .,1
169,zone,The white zone is for loading and unloading on...,0


In [17]:
df_final_dev["identifier"] = df_final_dev["word"]+"-"+df_final_dev["index"].astype(str)
df_final_train["identifier"] = df_final_train["word"]+"-"+df_final_train["index"].astype(str)
df_final_test["identifier"] = df_final_test["word"]+"-"+df_final_test["index"].astype(str)

In [18]:
df_odd = df_final_dev[1::2]
df_odd.columns = ["word2", "sent2", "index2", "identifier2"]
df_even = df_final_dev[::2]
df_even.columns = ["word1", "sent1", "index1", "identifier1"]
df_final_dev = pd.concat([df_even, df_odd], axis=1)

df_odd1 = df_final_train[1::2]
df_odd1.columns = ["word2", "sent2", "index2", "identifier2"]
df_even1 = df_final_train[::2]
df_even1.columns = ["word1", "sent1", "index1", "identifier1"]
df_final_train = pd.concat([df_even1, df_odd1], axis=1)

df_odd2 = df_final_test[1::2]
df_odd2.columns = ["word2", "sent2", "index2", "identifier2"]
df_even2 = df_final_test[::2]
df_even2.columns = ["word1", "sent1", "index1", "identifier1"]
df_final_test = pd.concat([df_even2, df_odd2], axis=1)

In [19]:
df_final_dev = df_final_dev[["word2", "sent2", "identifier2", "sent1", "identifier1", 'index1', 'index2']]
df_final_dev = df_final_dev.rename(columns={"word2":"lemma"})

df_final_train = df_final_train[["word2", "sent2", "identifier2", "sent1", "identifier1", 'index1', 'index2']]
df_final_train = df_final_train.rename(columns={"word2":"lemma"})

df_final_test = df_final_test[["word2", "sent2", "identifier2", "sent1", "identifier1", 'index1', 'index2']]
df_final_test = df_final_test.rename(columns={"word2":"lemma"})

In [20]:
df_final_dev["judgment"] = dev_label["judgment"].values.tolist()
df_final_train["judgment"] = train_label["judgment"].values.tolist()
df_final_test["judgment"] = test_label["judgment"].values.tolist()

In [21]:
df_final_dev = df_final_dev[['lemma', 'identifier2', 'identifier1', 'judgment']]
df_final_train = df_final_train[['lemma', 'identifier2', 'identifier1', 'judgment']]
df_final_test = df_final_test[['lemma', 'identifier2', 'identifier1', 'judgment']]

In [22]:
df_final_dev["annotator"] = " "
df_final_dev['comment'] = " "
df_final_train["annotator"] = " "
df_final_train['comment'] = " "
df_final_test["annotator"] = " "
df_final_test['comment'] = " "

In [23]:
df_final_dev = df_final_dev[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma"]]
df_final_train = df_final_train[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma"]]
df_final_test = df_final_test[["identifier1", "identifier2", "annotator", "judgment", "comment", "lemma"]]

In [24]:
df_final_dev = df_final_dev.reset_index(drop =True)
df_final_train = df_final_train.reset_index(drop =True)
df_final_test = df_final_test.reset_index(drop =True)

In [25]:
for i in list(df_final_dev["lemma"].value_counts().index):
        tmp_df = df_final_dev[df_final_dev["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/dev'+ "/"+i):
            os.mkdir('/content/WIC/dev'+ "/"+i)
        np.savetxt('/content/WIC/dev' +"/"+i+"/judgments.csv", numpy_df,fmt='%s', delimiter='\t')

In [26]:
for i in list(df_final_train["lemma"].value_counts().index):
        tmp_df = df_final_train[df_final_train["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/train'+ "/"+i):
            os.mkdir('/content/WIC/train'+ "/"+i)
        np.savetxt('/content/WIC/train' +"/"+i+"/judgments.csv", numpy_df,fmt='%s', delimiter='\t')

In [27]:
for i in list(df_final_test["lemma"].value_counts().index):
        tmp_df = df_final_test[df_final_test["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/test'+ "/"+i):
            os.mkdir('/content/WIC/test'+ "/"+i)
        np.savetxt('/content/WIC/test' +"/"+i+"/judgments.csv", numpy_df,fmt='%s', delimiter='\t')

In [28]:
ind1 = []
ind2 = []
for i in x:
    ind1.append(i.split('-')[0])
    ind2.append(i.split('-')[1])

ind_1 = []
ind_2 = []
for i in y:
    ind_1.append(i.split('-')[0])
    ind_2.append(i.split('-')[1])

indone = []
indtwo = []
for i in z:
    indone.append(i.split('-')[0])
    indtwo.append(i.split('-')[1])

In [29]:
df_dev.loc[:,'ind1'] = ind1
df_dev.loc[:,'ind2'] = ind2

df_train.loc[:,'ind_1'] = ind_1
df_train.loc[:,'ind_2'] = ind_2

df_test.loc[:,'indone'] = indone
df_test.loc[:,'indtwo'] = indtwo

In [30]:
df1 = df_dev[['lemma', 'pos', 'sent1', 'ind1']]
df1.columns =  ['lemma','pos', 'context_tokenized','ind']
df10 = df_train[['lemma', 'pos', 'sent1', 'ind_1']]
df10.columns =  ['lemma','pos', 'context_tokenized','ind']
df100 = df_test[['lemma', 'pos', 'sent1', 'indone']]
df100.columns =  ['lemma','pos', 'context_tokenized','ind']

In [31]:
df2 = df_dev[['lemma', 'pos', 'sent2', 'ind2']]
df2.columns =  ['lemma','pos', 'context_tokenized','ind']
df20 = df_train[['lemma', 'pos', 'sent2', 'ind_2']]
df20.columns =  ['lemma','pos', 'context_tokenized','ind']
df200 = df_test[['lemma', 'pos', 'sent2', 'indtwo']]
df200.columns =  ['lemma','pos', 'context_tokenized','ind']

In [32]:
df_final_dev_uses = pd.concat([df1, df2])
df_final_dev_uses = df_final_dev_uses.sort_values(by = ['lemma'], ascending = True)
df_final_train_uses = pd.concat([df10, df20])
df_final_train_uses= df_final_train_uses.sort_values(by = ['lemma'], ascending = True)
df_final_test_uses = pd.concat([df100, df200])
df_final_test_uses = df_final_test_uses.sort_values(by = ['lemma'], ascending = True)


In [33]:
df_final_dev_uses = df_final_dev_uses.reset_index(drop =True)
df_final_train_uses = df_final_train_uses.reset_index(drop =True)
df_final_test_uses = df_final_test_uses.reset_index(drop =True)

In [34]:
def get_indice(sent, indice):
  tag = ''
  indice = int(indice)
  tok = sent.split(" ")
  #print(tok)
  for i in range(len(tok)):
    if i == indice:
      tag = tok[i]
  return str(sent.find(tag))+":"+str(sent.find(tag)+len(tag))

In [35]:
df_final_dev_uses["indexes_target_token"] = df_final_dev_uses.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)
df_final_train_uses["indexes_target_token"] = df_final_train_uses.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)
df_final_test_uses["indexes_target_token"] = df_final_test_uses.apply(lambda x: get_indice(x.context_tokenized, x.ind), axis=1)

In [36]:
def get_indices_of_sent(sentence):
    return "0:"+str(len(sentence))

In [37]:
df_final_dev_uses["indexes_target_sentence"] = df_final_dev_uses["context_tokenized"].apply(get_indices_of_sent)
df_final_train_uses["indexes_target_sentence"] = df_final_train_uses["context_tokenized"].apply(get_indices_of_sent)
df_final_test_uses["indexes_target_sentence"] = df_final_test_uses["context_tokenized"].apply(get_indices_of_sent)

In [38]:
def get_len_tok(sentence):
  return "0:"+str(len(sentence.split(" ")))

In [39]:
df_final_dev_uses['indexes_target_sentence_tokenized'] = df_final_dev_uses["context_tokenized"].apply(get_len_tok)
df_final_train_uses['indexes_target_sentence_tokenized'] = df_final_train_uses["context_tokenized"].apply(get_len_tok)
df_final_test_uses['indexes_target_sentence_tokenized'] = df_final_test_uses["context_tokenized"].apply(get_len_tok)

In [40]:
lem = ''
def lemmatizr(sent):
  doc = nlp(sent)
  for i in doc:
    lem += i.lemma_
    return lem

In [41]:
df_final_dev_uses['context_lemmatized'] = df_final_dev_uses["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [42]:
df_final_train_uses['context_lemmatized'] = df_final_train_uses["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))
df_final_test_uses['context_lemmatized'] = df_final_test_uses["context_tokenized"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [43]:
df_final_dev_uses['context_pos'] = df_final_dev_uses["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))
df_final_train_uses['context_pos'] = df_final_train_uses["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))
df_final_test_uses['context_pos'] = df_final_test_uses["context_tokenized"].apply(lambda row: " ".join([w.pos_ for w in nlp(row)]))

In [44]:
df_final_dev_uses['date'] = " "
df_final_dev_uses['grouping'] = 1
df_final_dev_uses['description'] = " "
df_final_dev_uses['context'] = df_final_dev_uses['context_tokenized']
df_final_dev_uses['identifier'] = " "
df_final_dev_uses['indexes_target_token_tokenized'] = df_final_dev_uses['ind']

df_final_train_uses['date'] = " "
df_final_train_uses['grouping'] = 1
df_final_train_uses['description'] = " "
df_final_train_uses['context'] = df_final_train_uses['context_tokenized']
df_final_train_uses['identifier'] = " "
df_final_train_uses['indexes_target_token_tokenized'] = df_final_train_uses['ind']

df_final_test_uses['date'] = " "
df_final_test_uses['grouping'] = 1
df_final_test_uses['description'] = " "
df_final_test_uses['context'] = df_final_test_uses['context_tokenized']
df_final_test_uses['identifier'] = " "
df_final_test_uses['indexes_target_token_tokenized'] = df_final_test_uses['ind']

In [45]:
final_df_dev = df_final_dev_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]
final_df_train = df_final_train_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]
final_df_test = df_final_test_uses[['lemma', 'pos', 'date', 'grouping', 'identifier', 'description', 'context', 'indexes_target_token', 'indexes_target_sentence', 'context_tokenized', 'indexes_target_token_tokenized', 'indexes_target_sentence_tokenized', 'context_lemmatized', 'context_pos']]

In [46]:
for i in list(final_df_dev["lemma"].value_counts().index):
        tmp_df = final_df_dev[final_df_dev["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/dev'+"/"+i):
            os.mkdir('/content/WIC/dev'+"/"+i)
        np.savetxt('/content/WIC/dev'+"/"+i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')

In [47]:
for i in list(final_df_train["lemma"].value_counts().index):
        tmp_df = final_df_train[final_df_train["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/train'+"/"+i):
            os.mkdir('/content/WIC/train'+"/"+i)
        np.savetxt('/content/WIC/train'+"/"+i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')

In [48]:
for i in list(final_df_test["lemma"].value_counts().index):
        tmp_df = final_df_test[final_df_test["lemma"]==i]
        numpy_df = tmp_df.to_numpy()
        header = list(tmp_df.columns)
        numpy_df = np.vstack([header, numpy_df])
        if not os.path.exists('/content/WIC/test'+"/"+i):
            os.mkdir('/content/WIC/test'+"/"+i)
        np.savetxt('/content/WIC/test'+"/"+i+"/uses.csv", numpy_df,fmt='%s', delimiter='\t')

In [49]:
final_df_train

,lemma,pos,date,grouping,identifier,description,context,indexes_target_token,indexes_target_sentence,context_tokenized,indexes_target_token_tokenized,indexes_target_sentence_tokenized,context_lemmatized,context_pos
0,ability,N,,1,,,This phone has the ability to have its softwar...,19:26,0:69,This phone has the ability to have its softwar...,4,0:12,this phone have the ability to have its softwa...,DET NOUN VERB DET NOUN PART AUX PRON NOUN VERB...
1,ability,N,,1,,,This wood has the ability to fight off insects...,18:25,0:92,This wood has the ability to fight off insects...,4,0:19,this wood have the ability to fight off insect...,DET NOUN VERB DET NOUN PART VERB ADP NOUN PUNC...
2,absence,N,,1,,,Absence makes the heart grow fonder .,0:7,0:37,Absence makes the heart grow fonder .,0,0:7,absence make the heart grow fonder .,NOUN VERB DET NOUN VERB NOUN PUNCT
3,absence,N,,1,,,Absence makes the heart grow fonder .,0:7,0:37,Absence makes the heart grow fonder .,0,0:7,absence make the heart grow fonder .,NOUN VERB DET NOUN VERB NOUN PUNCT
4,absence,N,,1,,,He visited during my absence .,21:28,0:30,He visited during my absence .,4,0:6,he visit during my absence .,PRON VERB ADP PRON NOUN PUNCT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10851,yes,N,,1,,,Was that a yes ?,11:14,0:16,Was that a yes ?,3,0:5,be that a yes ?,AUX PRON DET INTJ PUNCT
10852,youth,N,,1,,,Her youth and beauty is what attracted him to ...,4:9,0:51,Her youth and beauty is what attracted him to ...,1,0:11,her youth and beauty be what attract he to she .,PRON NOUN CCONJ NOUN AUX PRON VERB PRON ADP PR...
10853,youth,N,,1,,,"Make the most of your youth , it will not last...",22:27,0:56,"Make the most of your youth , it will not last...",5,0:13,"make the most of your youth , it will not last...",VERB DET ADJ ADP PRON NOUN PUNCT PRON AUX PART...
10854,youth,N,,1,,,"I made many mistakes in my youth , but learned...",27:32,0:62,"I made many mistakes in my youth , but learned...",6,0:14,"I make many mistake in my youth , but learn fr...",PRON VERB ADJ NOUN ADP PRON NOUN PUNCT CCONJ V...


In [50]:
#c = final_df.context.tolist()

In [51]:
#y = []
#for m in c:
 # y.append(m.split(" "))

In [52]:
#lengthh = []
#for i in y:
 #   lengthh.append(len(i))

In [53]:
#final_df.loc[:,'lengthh'] = lengthh

In [54]:
#d = final_df.context_pos.tolist()

In [55]:
#z = []
#for m in d:
 # z.append(m.split(" "))

In [56]:
#lengthh2 = []
#for i in z:
 #   lengthh2.append(len(i))

In [57]:
#final_df.loc[:,'lengthh2'] = lengthh2